<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-представленных-данных" data-toc-modified-id="Описание-представленных-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание представленных данных</a></span><ul class="toc-item"><li><span><a href="#train.csv" data-toc-modified-id="train.csv-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>train.csv</a></span></li><li><span><a href="#questions.csv" data-toc-modified-id="questions.csv-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>questions.csv</a></span></li><li><span><a href="#lectures.csv" data-toc-modified-id="lectures.csv-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>lectures.csv</a></span></li></ul></li><li><span><a href="#Разберем-данные-таблиц" data-toc-modified-id="Разберем-данные-таблиц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разберем данные таблиц</a></span></li><li><span><a href="#Общие-выводы" data-toc-modified-id="Общие-выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общие выводы</a></span></li></ul></div>

## Описание представленных данных

**Ссылка для скачивания данных:**<br>
https://drive.google.com/drive/folders/1k0k3d-WO9kYqp0q9QG1WcmLn6mWWs0Zd

**Файлы с данными:**
<li>train.csv</li>
<li>questions.csv</li>
<li>lectures.csv</li>

Подключим Google Drive к колаб ноутбуку

**Загрузка библиотек**

Для начала отключим предупреждения

In [13]:
import warnings

# Отключение всех предупреждений
warnings.filterwarnings("ignore")

In [14]:
!pip install polars
import polars as pl


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install altair


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# Путь к файлу в корневой директории
file_path_train = r'C:/Users/Igor/Downloads/train.csv'
file_path_questions = r'C:/Users/Igor/Downloads/questions.csv'
file_path_lectures = r'C:/Users/Igor/Downloads/lectures.csv'

# Чтение CSV файла
train_df = pl.read_csv(file_path_train)
questions_df = pl.read_csv(file_path_questions)
lectures_df = pl.read_csv(file_path_lectures)

### train.csv

In [17]:
# Узнаем размер DataFrame
size_before = train_df.estimated_size()
size_before_mb = size_before / (1024 * 1024)
print(f"Размер DataFrame до приведения типов: {size_before_mb:.2f} МБ")

Размер DataFrame до приведения типов: 6987.14 МБ


**Посмотрим названия колонок**

In [18]:
schema_info = train_df.schema
print(schema_info)

Schema([('row_id', Int64), ('timestamp', Int64), ('user_id', Int64), ('content_id', Int64), ('content_type_id', Int64), ('task_container_id', Int64), ('user_answer', Int64), ('answered_correctly', Int64), ('prior_question_elapsed_time', Float64), ('prior_question_had_explanation', Boolean)])


Во всех колонках, кроме 'prior_question_had_explanation' использованы 64-битные типы данных, посмотрим, что за данные, и можно ли от них отойти, чтобы сэкономить пространство

In [19]:
# Вывод последних 10 строк train
print(train_df.tail(10))

shape: (10, 10)
┌───────────┬───────────┬────────────┬────────────┬───┬─────────────┬────────────────┬────────────────┬────────────────┐
│ row_id    ┆ timestamp ┆ user_id    ┆ content_id ┆ … ┆ user_answer ┆ answered_corre ┆ prior_question ┆ prior_question │
│ ---       ┆ ---       ┆ ---        ┆ ---        ┆   ┆ ---         ┆ ctly           ┆ _elapsed_time  ┆ _had_explanati │
│ i64       ┆ i64       ┆ i64        ┆ i64        ┆   ┆ i64         ┆ ---            ┆ ---            ┆ on             │
│           ┆           ┆            ┆            ┆   ┆             ┆ i64            ┆ f64            ┆ ---            │
│           ┆           ┆            ┆            ┆   ┆             ┆                ┆                ┆ bool           │
╞═══════════╪═══════════╪════════════╪════════════╪═══╪═════════════╪════════════════╪════════════════╪════════════════╡
│ 101230322 ┆ 428415725 ┆ 2147482888 ┆ 5005       ┆ … ┆ 1           ┆ 0              ┆ 30000.0        ┆ true           │
│ 101230323 ┆ 42

Видно, что row id может уместиться в int32, убедимся, что макс значения в  графах row_id, content_id, user_answer и prior_question_elapsed_time моместятся в уменьшенные версии типов данных и сразу же преобразуем

In [20]:
# Определение типов данных для каждого столбца
def determine_type(min_value, max_value):
    if min_value >= -128 and max_value <= 127:
        return pl.Int8
    elif min_value >= -32768 and max_value <= 32767:
        return pl.Int16
    elif min_value >= -2147483648 and max_value <= 2147483647:
        return pl.Int32
    else:
        return pl.Int64

# Преобразование типов данных
train_df = train_df.with_columns([
    pl.col('row_id').cast(determine_type(train_df['row_id'].min(), train_df['row_id'].max())),
    pl.col('content_id').cast(determine_type(train_df['content_id'].min(), train_df['content_id'].max())),
    pl.col('user_answer').cast(determine_type(train_df['user_answer'].min(), train_df['user_answer'].max())),
    pl.col('prior_question_elapsed_time').cast(pl.Float32)  # Преобразование в Float32 для экономии памяти
])

# Вывод информации о типах данных
print("\nИнформация о типах данных после приведения:")
print(train_df.schema)


Информация о типах данных после приведения:
Schema([('row_id', Int32), ('timestamp', Int64), ('user_id', Int64), ('content_id', Int16), ('content_type_id', Int64), ('task_container_id', Int64), ('user_answer', Int8), ('answered_correctly', Int64), ('prior_question_elapsed_time', Float32), ('prior_question_had_explanation', Boolean)])


Из-за особенностей отображения мы не видим значения в колонках 'content_type_id' 'task_container_id', выведем их отдельно

In [21]:
# Выбор столбцов 'content_type_id' и 'task_container_id'
selected_columns = train_df.select([
    pl.col('content_type_id'),
    pl.col('task_container_id')
])

# Вывод первых нескольких строк выбранных столбцов
print(selected_columns.head(10))


shape: (10, 2)
┌─────────────────┬───────────────────┐
│ content_type_id ┆ task_container_id │
│ ---             ┆ ---               │
│ i64             ┆ i64               │
╞═════════════════╪═══════════════════╡
│ 0               ┆ 1                 │
│ 0               ┆ 2                 │
│ 0               ┆ 0                 │
│ 0               ┆ 3                 │
│ 0               ┆ 4                 │
│ 0               ┆ 5                 │
│ 0               ┆ 6                 │
│ 0               ┆ 7                 │
│ 0               ┆ 8                 │
│ 0               ┆ 9                 │
└─────────────────┴───────────────────┘


In [22]:
# Вывод последних нескольких строк выбранных столбцов
print(selected_columns.tail(10))

shape: (10, 2)
┌─────────────────┬───────────────────┐
│ content_type_id ┆ task_container_id │
│ ---             ┆ ---               │
│ i64             ┆ i64               │
╞═════════════════╪═══════════════════╡
│ 0               ┆ 17                │
│ 0               ┆ 18                │
│ 0               ┆ 19                │
│ 0               ┆ 20                │
│ 0               ┆ 21                │
│ 0               ┆ 22                │
│ 0               ┆ 23                │
│ 0               ┆ 24                │
│ 0               ┆ 25                │
│ 0               ┆ 26                │
└─────────────────┴───────────────────┘


In [23]:
# Вывод уникальных значений для столбца 'content_type_id'
unique_content_type_id = train_df['content_type_id'].unique()
print("Уникальные значения в столбце 'content_type_id':")
print(unique_content_type_id)

# Вывод уникальных значений для столбца 'task_container_id'
unique_task_container_id = train_df['task_container_id'].unique()
print("\nУникальные значения в столбце 'task_container_id':")
print(unique_task_container_id)


Уникальные значения в столбце 'content_type_id':
shape: (2,)
Series: 'content_type_id' [i64]
[
	0
	1
]

Уникальные значения в столбце 'task_container_id':
shape: (10_000,)
Series: 'task_container_id' [i64]
[
	0
	1
	2
	3
	4
	…
	9995
	9996
	9997
	9998
	9999
]


**Видим, что уникальных значений в столбце content_type_id 2 - это 0 и 1, а в столбце task_container_id 1000 можно их прееобразовать в другой тип данных для экономии**


In [24]:
train_df = train_df.with_columns([
    pl.col('content_type_id').cast(pl.Boolean),  # Преобразование в Boolean
    pl.col('task_container_id').cast(pl.Int16)    # Преобразование в Int16
])

Теперь выведем все типы в таблице

In [25]:
schema_info = train_df.schema
print(schema_info)

Schema([('row_id', Int32), ('timestamp', Int64), ('user_id', Int64), ('content_id', Int16), ('content_type_id', Boolean), ('task_container_id', Int16), ('user_answer', Int8), ('answered_correctly', Int64), ('prior_question_elapsed_time', Float32), ('prior_question_had_explanation', Boolean)])


Посмотрим, сколько теперь файл занимает памяти

In [26]:
# Узнаем размер DataFrame
size_after = train_df.estimated_size()
size_after_mb = size_after / (1024 * 1024)
print(f"Размер DataFrame до приведения типов: {size_after_mb:.2f} МБ")

Размер DataFrame до приведения типов: 3620.28 МБ


Ура! Размер датафрейма сократился в 2 раза, теперь можно с ним что-то делать.

In [27]:
#посмотрим
train_df.describe()

statistic,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.01230332e8,1.01230332e8,1.01230332e8,1.01230332e8,1.01230332e8,1.01230332e8,1.01230332e8,1.01230332e8,9.8878794e7,1.00837826e8
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.351538e6,392506.0
"""mean""",5.0615e7,7.7036e9,1.0767e9,5219.604757,0.019352,904.062366,1.376123,0.625164,25423.810547,0.889404
"""std""",2.9223e7,1.1593e10,6.1972e8,3866.35894,null,1358.302181,1.192896,0.522531,19948.146484,null
"""min""",0.0,0.0,115.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0
"""25%""",2.5307583e7,5.24343582e8,5.40811564e8,2063.0,null,104.0,0.0,0.0,16000.0,null
"""50%""",5.0615166e7,2.6742e9,1.0718e9,5026.0,null,382.0,1.0,1.0,21000.0,null
"""75%""",7.5922748e7,9.9246e9,1.6157e9,7425.0,null,1094.0,3.0,1.0,29666.0,null
"""max""",1.01230331e8,8.7426e10,2.1475e9,32736.0,1.0,9999.0,3.0,1.0,300000.0,1.0


И сделаем перевод названий колонок

**row_id**: (int64) Идентификатор строки.<br>
**timestamp**: (int64) Время в миллисекундах между этим взаимодействием пользователя и первым событием завершения от этого пользователя.<br>
**user_id**: (int32) Идентификатор пользователя.<br>
**content_id**: (int16) Идентификатор взаимодействия пользователя.<br>
**content_type_id**: (int8) 0, если событие было вопросом, заданным пользователю, 1, если событие было просмотром лекции пользователем.<br>
**task_container_id**: (int16) Идентификатор пакета заданий или лекций. Например, пользователь может увидеть три вопроса подряд, прежде чем увидеть объяснения к ним. Все три вопроса будут иметь один и тот же task_container_id.<br>
**user_answer**: (int8) Ответ пользователя на вопрос, если таковой имеется. Читайте -1 как null для лекций.<br>
**answered_correctly**: (int8) Если пользователь ответил правильно. Читайте -1 как null для лекций.<br>
**prior_question_elapsed_time**: (float32) Среднее время в миллисекундах, которое пользователь потратил на ответы на каждый вопрос в предыдущем пакете вопросов, игнорируя любые лекции между ними. Имеет значение null для первого пакета вопросов или лекции пользователя. Обратите внимание, что время — это среднее время, которое пользователь потратил на решение каждого вопроса в предыдущем пакете.<br>
**prior_question_had_explanation**: (bool) Было ли пользователю показано объяснение и правильные ответы после ответа на предыдущий пакет вопросов, игнорируя любые лекции между ними. Значение совместно используется в рамках одного пакета вопросов и имеет значение null для первого пакета вопросов или лекции пользователя. Обычно первые несколько вопросов, которые видит пользователь, являются частью диагностического теста на адаптацию, где он не получает никакой обратной связи.

### questions.csv


In [28]:
#посмотрим
questions_df.describe()

statistic,question_id,bundle_id,correct_answer,part,tags
str,f64,f64,f64,f64,str
"""count""",13523.0,13523.0,13523.0,13523.0,"""13522"""
"""null_count""",0.0,0.0,0.0,0.0,"""1"""
"""mean""",6761.0,6760.510907,1.455298,4.264956,null
"""std""",3903.89818,3903.857783,1.149707,1.652553,null
"""min""",0.0,0.0,0.0,1.0,"""1"""
"""25%""",3381.0,3381.0,0.0,3.0,null
"""50%""",6761.0,6761.0,1.0,5.0,null
"""75%""",10142.0,10140.0,3.0,5.0,null
"""max""",13522.0,13522.0,3.0,7.0,"""98 97 77 135 162"""


***Переведем названия колонок***

**question_id**: внешний ключ для столбца content_id в train/test, когда тип контента — вопрос (0).<br>
**bundle_id**: код, определяющий, какие вопросы показываются вместе.<br>
**correct_answer**: правильный ответ на вопрос. Может быть сравнен со столбцом user_answer в train, чтобы проверить, был ли пользователь прав.<br>
**part**: соответствующий раздел теста TOEIC.<br>
**tags**: один или несколько детальных кодов тегов для вопроса. Значение тегов не будет предоставлено, но эти коды достаточны для кластеризации вопросов.

In [29]:
# Вывод первых нескольких строк questions
print(questions_df.head())

shape: (5, 5)
┌─────────────┬───────────┬────────────────┬──────┬────────────────┐
│ question_id ┆ bundle_id ┆ correct_answer ┆ part ┆ tags           │
│ ---         ┆ ---       ┆ ---            ┆ ---  ┆ ---            │
│ i64         ┆ i64       ┆ i64            ┆ i64  ┆ str            │
╞═════════════╪═══════════╪════════════════╪══════╪════════════════╡
│ 0           ┆ 0         ┆ 0              ┆ 1    ┆ 51 131 162 38  │
│ 1           ┆ 1         ┆ 1              ┆ 1    ┆ 131 36 81      │
│ 2           ┆ 2         ┆ 0              ┆ 1    ┆ 131 101 162 92 │
│ 3           ┆ 3         ┆ 0              ┆ 1    ┆ 131 149 162 29 │
│ 4           ┆ 4         ┆ 3              ┆ 1    ┆ 131 5 162 38   │
└─────────────┴───────────┴────────────────┴──────┴────────────────┘


### lectures.csv
метаданные для лекций, просмотренных пользователями в процессе обучения.


In [30]:
#посмотрим общую информацию о данных в датафрейме
lectures_df.describe()

statistic,lecture_id,tag,part,type_of
str,f64,f64,f64,str
"""count""",418.0,418.0,418.0,"""418"""
"""null_count""",0.0,0.0,0.0,"""0"""
"""mean""",16983.401914,94.480861,4.267943,null
"""std""",9426.16466,53.586487,1.872424,null
"""min""",89.0,0.0,1.0,"""concept"""
"""25%""",8977.0,50.0,2.0,null
"""50%""",17199.0,95.0,5.0,null
"""75%""",24908.0,140.0,6.0,null
"""max""",32736.0,187.0,7.0,"""starter"""


In [31]:
# Вывод первых нескольких строк questions
print(lectures_df.head())

shape: (5, 4)
┌────────────┬─────┬──────┬──────────────────┐
│ lecture_id ┆ tag ┆ part ┆ type_of          │
│ ---        ┆ --- ┆ ---  ┆ ---              │
│ i64        ┆ i64 ┆ i64  ┆ str              │
╞════════════╪═════╪══════╪══════════════════╡
│ 89         ┆ 159 ┆ 5    ┆ concept          │
│ 100        ┆ 70  ┆ 1    ┆ concept          │
│ 185        ┆ 45  ┆ 6    ┆ concept          │
│ 192        ┆ 79  ┆ 5    ┆ solving question │
│ 317        ┆ 156 ┆ 5    ┆ solving question │
└────────────┴─────┴──────┴──────────────────┘


**lecture_id**: внешний ключ для столбца content_id в train/test, когда тип контента — лекция (1).<br>
**part**: код верхнего уровня для категории лекции.<br>
**tag**: один код тега для лекции. Значение тегов не будет предоставлено, но эти коды достаточны для кластеризации лекций.<br>
**type_of**: краткое описание основной цели лекции.

**Преобразованием данных в файлах questions.csv и lectures.csv мы заниматься не будем, т.к. они и так нормально считываются**

## Разберем данные таблиц

In [32]:
unique_users_count = train_df['user_id'].n_unique()
print(f"\nКоличество уникальных пользователей: {unique_users_count}")


Количество уникальных пользователей: 393656


**Количество вопросов и лекций**

In [33]:
questions_count = questions_df.shape[0]
lectures_count = lectures_df.shape[0]
print(f"Количество вопросов: {questions_count}")
print(f"Количество лекций: {lectures_count}")

Количество вопросов: 13523
Количество лекций: 418


**Распределение ответов пользователей**

In [34]:
user_answers_distribution = train_df['user_answer'].value_counts()
print("\nРаспределение ответов пользователей:")
print(user_answers_distribution)


Распределение ответов пользователей:
shape: (5, 2)
┌─────────────┬──────────┐
│ user_answer ┆ count    │
│ ---         ┆ ---      │
│ i8          ┆ u32      │
╞═════════════╪══════════╡
│ 1           ┆ 26990007 │
│ 3           ┆ 26084784 │
│ 2           ┆ 18010020 │
│ 0           ┆ 28186489 │
│ -1          ┆ 1959032  │
└─────────────┴──────────┘


**Распределение правильных ответов**

In [35]:
correct_answers_distribution = train_df['answered_correctly'].value_counts()
print("\nРаспределение правильных ответов:")
print(correct_answers_distribution)


Распределение правильных ответов:
shape: (3, 2)
┌────────────────────┬──────────┐
│ answered_correctly ┆ count    │
│ ---                ┆ ---      │
│ i64                ┆ u32      │
╞════════════════════╪══════════╡
│ 0                  ┆ 34026673 │
│ 1                  ┆ 65244627 │
│ -1                 ┆ 1959032  │
└────────────────────┴──────────┘


**Распределение типов контента**

In [36]:
content_type_distribution = train_df['content_type_id'].value_counts()
print("\nРаспределение типов контента:")
print(content_type_distribution)


Распределение типов контента:
shape: (2, 2)
┌─────────────────┬──────────┐
│ content_type_id ┆ count    │
│ ---             ┆ ---      │
│ bool            ┆ u32      │
╞═════════════════╪══════════╡
│ true            ┆ 1959032  │
│ false           ┆ 99271300 │
└─────────────────┴──────────┘


False означает, что пользователю был задан вопрос. True означает, что пользователь смотрел лекцию.

Посчитаем количество уникальных идентификаторов content_id

In [37]:
# Подсчет уникальных идентификаторов content_id
total_unique_content_ids = train_df['content_id'].unique().shape[0]

# Фильтрация по content_type_id == False и подсчет уникальных content_id
questions_unique_content_ids = train_df.filter(pl.col('content_type_id') == False)['content_id'].unique().shape[0]

# Вывод результата
print(f'Всего {total_unique_content_ids} уникальных идентификаторов, из которых {questions_unique_content_ids} - вопросы.')

Всего 13782 уникальных идентификаторов, из которых 13523 - вопросы.


Импортируем библиотеки для отображения информации

In [38]:
import polars as pl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import altair as alt

In [39]:
# Подсчет частоты content_id
cids = train_df.group_by('content_id').agg(pl.count().alias('count')).sort('count').head(30)

# Построение графика с использованием altair
chart = alt.Chart(cids).mark_bar().encode(
    x=alt.X('content_id:N', sort='-y', title='Content ID'),
    y=alt.Y('count:Q', title='Count')
).properties(
    title="30 самых популярных content_id"
)

chart.configure_axis(
    labelAngle=90
).configure_title(
    fontSize=16
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_view(
    strokeWidth=0
).display()

alt.Chart(...)

In [40]:
task_container_id_unique = train_df['task_container_id'].unique()
print(f'Всего {len(task_container_id_unique)} уникальных пакетов вопросов или лекций.')

Всего 10000 уникальных пакетов вопросов или лекций.


In [41]:
train_df['user_answer'].value_counts()

user_answer,count
i8,u32
1,26990007
-1,1959032
0,28186489
2,18010020
3,26084784


prior_question_had_explanation содержит 100,837,826 записей и 392,506 пропущенных значений.
Среднее значение prior_question_had_explanation составляет около 0.889404, что указывает на то, что в 88.9% случаев учащиеся знакомились с объяснением.

prior_question_elapsed_time содержит 98,878,794 записей (2,351,538 пропущенных значений). Среднее значение prior_question_elapsed_time составляет около 25423.810547 миллисекунд. Что означает, что в среднем пользователь тратил 25 секунд на вопрос, если 

**Рассчитаем зависимость между prior_question_had_explanation и answered_correctly**


In [42]:
# Сгруппируем данные по answered_correctly и prior_question_had_explanation
result = train_df.group_by(['answered_correctly', 'prior_question_had_explanation']).agg(
    pl.count().alias('count')
)

# Вывести результат
print(result)

shape: (7, 3)
┌────────────────────┬────────────────────────────────┬──────────┐
│ answered_correctly ┆ prior_question_had_explanation ┆ count    │
│ ---                ┆ ---                            ┆ ---      │
│ i64                ┆ bool                           ┆ u32      │
╞════════════════════╪════════════════════════════════╪══════════╡
│ 1                  ┆ false                          ┆ 4602399  │
│ 1                  ┆ null                           ┆ 266940   │
│ 1                  ┆ true                           ┆ 60375288 │
│ 0                  ┆ null                           ┆ 125566   │
│ 0                  ┆ false                          ┆ 4590835  │
│ 0                  ┆ true                           ┆ 29310272 │
│ -1                 ┆ false                          ┆ 1959032  │
└────────────────────┴────────────────────────────────┴──────────┘


In [43]:
# Построение графика с использованием altair
chart = alt.Chart(result).mark_bar().encode(
    x=alt.X('answered_correctly:N', title='Answered Correctly'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('prior_question_had_explanation:N', title='Had Explanation')
).properties(
    title="Frequency of Explanations After Correct/Incorrect Answers"
)

chart.configure_axis(
    labelAngle=0
).configure_title(
    fontSize=16
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_view(
    strokeWidth=0
).display()

alt.Chart(...)

**Вывод 1**:

Анализ данных показывает, что наличие разъяснения на предыдущий вопрос значительно влияет на правильность ответа на следующий вопрос. (60,375,288 раз против 4,602,399 раз без разъяснения). Учащиеся чаще отвечают правильно, когда они получали разъяснения, и чаще ошибаются, когда разъяснения не было.

Посмотрим зависимость правильности ответов от общего времени

In [44]:
result = train_df.filter(
    # отрезаем выбросы выше 90-го перцентиля
    (pl.col('timestamp') < pl.col('timestamp').quantile(0.9)) &
    # отрезаем выбросы ниже относительно малого значения
    (pl.col('timestamp') > 300000000)
).group_by(
    # группируем данные по времени, сгладив результаты
    (pl.col('timestamp') // 250000000).alias('time'),
).agg(
    # для аггрегации возьмем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
).sort('time')

# Построение линейного графика с использованием altair
chart = alt.Chart(result).mark_line().encode(
    x=alt.X('time:Q', title='Time'),
    y=alt.Y('mean:Q', title='Mean Answered Correctly')
).properties(
    title="Mean Answered Correctly by Time"
)

chart.configure_axis(
    labelAngle=0
).configure_title(
    fontSize=16
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_view(
    strokeWidth=0
).display()

alt.Chart(...)

In [45]:
# Фильтрация данных
filtered_df = train_df.filter(
    pl.col('timestamp').is_not_null() &
    pl.col('answered_correctly').is_not_null()
)

# Расчет корреляции
correlation = filtered_df.select(
    pl.corr('timestamp', 'answered_correctly')
).to_series().item()

# Вывод результата
print(f"Коэффициент корреляции Пирсона: {correlation}")

Коэффициент корреляции Пирсона: 0.025452223850250012


Как видим, корелляция времени и правильных ответов слабая

Для большей информативности посмотрим, как зависит среднее количество правильных ответов от полного времени обучения.

In [46]:
result = train_df.group_by(
    pl.col('user_id')
).agg(
    # количество строк
    pl.col('row_id').count().alias('row_count'),
    # максимальное значение времени
    pl.col('timestamp').max().alias('time'),
    # среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('answered_correctly_mean')
).filter(
    # уберем из выборки результаты с малым количеством строк
    pl.col('row_count') > pl.col('row_count').quantile(0.1)
).group_by(
    # группируем по сглаженному времени
    (pl.col('time') // 1000000000).alias('time_group')
).agg(
    # аггрегируем среднее по правильным ответам
    pl.col('answered_correctly_mean').mean().alias('mean_answered_correctly')
).sort('time_group')

print(result)

shape: (88, 2)
┌────────────┬─────────────────────────┐
│ time_group ┆ mean_answered_correctly │
│ ---        ┆ ---                     │
│ i64        ┆ f64                     │
╞════════════╪═════════════════════════╡
│ 0          ┆ 0.499037                │
│ 1          ┆ 0.572325                │
│ 2          ┆ 0.576242                │
│ 3          ┆ 0.577843                │
│ 4          ┆ 0.579758                │
│ …          ┆ …                       │
│ 83         ┆ 0.634343                │
│ 84         ┆ 0.669204                │
│ 85         ┆ 0.651755                │
│ 86         ┆ 0.624567                │
│ 87         ┆ 0.705971                │
└────────────┴─────────────────────────┘


In [47]:
# Построение графика с использованием altair
chart = alt.Chart(result).mark_line().encode(
    x=alt.X('time_group:Q', title='Time Group'),
    y=alt.Y('mean_answered_correctly:Q', title='Mean Answered Correctly')
).properties(
    title="Mean Answered Correctly by Time Group"
)

chart.configure_axis(
    labelAngle=0
).configure_title(
    fontSize=16
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_view(
    strokeWidth=0
).display()

alt.Chart(...)

**Вывод 2:**
Среднее количество правильных ответов (mean_answered_correctly) постепенно увеличивается с течением времени. Т.е. чем дольше пользователи, тем чаще дают правильные ответы.

Попробуем найти зависимость правильных ответов от затраченного на вопрос времени в пределах 95% доверительного интервала

In [48]:
# Фильтрация данных
filtered_df = train_df.filter(
    (pl.col('prior_question_elapsed_time') < pl.col('prior_question_elapsed_time').quantile(0.95)) &
    (pl.col('prior_question_elapsed_time') > pl.col('prior_question_elapsed_time').quantile(0.05))
)

# Выбор и срез данных
selected_df = filtered_df.select(
    pl.col('prior_question_elapsed_time').slice(1, None),
    pl.col('answered_correctly').slice(0, pl.col('answered_correctly').count() - 1)
)

# Группировка и агрегация данных
result = selected_df.group_by(
    (pl.col('prior_question_elapsed_time') // 1000).alias('time')
).agg(
    pl.col('answered_correctly').mean().alias('answered_correctly_mean')
).sort('time')

# Построение графика с использованием altair
chart = alt.Chart(result).mark_line().encode(
    x='time:Q',
    y='answered_correctly_mean:Q'
).properties(
    title='Mean Answered Correctly by Time'
)

# Отображение графика
chart.show()

alt.Chart(...)

**Вывод 3**: Из графика сдедует, что процент правильных ответов выше в первые 20 секунд. Дальше этот показатель постепенно снижается.

**Изучим зависимость полученной оценки от числа посещенных занятий**

In [64]:
# Группируем данные по пользователям
result = (train_df.group_by('user_id')
    # Посчитаем необходимые аггрегаты
    .agg(
        # Количество строк
        pl.col('row_id').count().alias('rows'),
        # Количество лекций
        (pl.col('content_type_id') == 1).sum().alias('lectures'),
        # Количество вопросов
        (pl.col('content_type_id') == 0).sum().alias('questions'),
        # Количество правильных ответов
        (pl.col('answered_correctly') == 1).sum().alias('correct'),
    ).select(
        # Количество лекций
        pl.col('lectures'),
        # Считаем процент правильных ответов
        (pl.col('correct') / pl.col('questions') * 100).alias('correct_%')
    ).group_by(
        # Группируем по количеству лекций
        pl.col('lectures')
    ).agg(
        # Аггрегируем среднее по правильным ответам
        pl.col('correct_%').mean().alias('mean_correct_%')
    ).sort('lectures')
    .drop_nulls()  # Удаляем строки с пропущенными значениями
)

# Выводим результат
print(result)

shape: (293, 2)
┌──────────┬────────────────┐
│ lectures ┆ mean_correct_% │
│ ---      ┆ ---            │
│ u32      ┆ f64            │
╞══════════╪════════════════╡
│ 0        ┆ 50.367738      │
│ 1        ┆ 56.934262      │
│ 2        ┆ 59.332606      │
│ 3        ┆ 60.931632      │
│ 4        ┆ 61.652804      │
│ …        ┆ …              │
│ 385      ┆ 78.851472      │
│ 386      ┆ 76.006502      │
│ 389      ┆ 71.222831      │
│ 395      ┆ 73.315879      │
│ 397      ┆ 72.360047      │
└──────────┴────────────────┘


In [65]:
# Для наглядности построим график
chart = alt.Chart(result).mark_line().encode(
    x=alt.X('lectures:Q', title='Number of Lectures'),
    y=alt.Y('mean_correct_%:Q', title='Mean Correct Answers (%)')
).properties(
    title='Mean Correct Answers by Number of Lectures'
)

# Отображение графика
chart.show()

alt.Chart(...)

Видно, что процент средних ответов растет, по после 320 лекций большой разброс в данных (это может быть связано с тем, что после этого количества лекций у вас становится меньше данных, и средние значения становятся менее стабильными). Применим сглаживание с помощью логарифмической функции.

In [71]:
# Применение скользящего среднего для сглаживания данных
result = result.with_columns(
    pl.col('mean_correct_%').rolling_mean(window_size=10).alias('mean_correct_%_smooth')
)

# Построение графика с использованием altair
chart = alt.Chart(result).mark_line().encode(
    x=alt.X('lectures:Q', title='Number of Lectures'),
    y=alt.Y('mean_correct_%_smooth:Q', title='Mean Correct Answers (%)')
).properties(
    title='Mean Correct Answers by Number of Lectures (Smoothed)'
)

# Отображение графика
chart.show()

alt.Chart(...)

**Вывод 4**: Есть прямая зависимость числа посещенных занятий и полученной оценки

## Общие выводы


Таким образом, мы изучили данные представленных таблиц и на основании полученных данных сделали 4 вывода.

Вывод 1:
Наличие разъяснения на предыдущий вопрос значительно влияет на правильность ответа на следующий вопрос. (60,375,288 раз против 4,602,399 раз без разъяснения). Учащиеся чаще отвечают правильно, когда они получали разъяснения, и чаще ошибаются, когда разъяснения не было.

Вывод 2: 
Среднее количество правильных ответов (mean_answered_correctly) постепенно увеличивается с течением времени. Т.е. чем дольше пользователи обучаются, тем чаще дают правильные ответы.

Вывод 3:
Процент правильных ответов на вопросы выше в первые 20 секунд. Дальше этот показатель постепенно снижается.

Вывод 4: Есть прямая зависимость числа посещенных занятий и полученной оценки